In [133]:
import numpy as np
from scipy.spatial import Voronoi

# ДОРАБОТАТЬ ДЛЯ ПРОИЗВОЛЬНОГО ЧИСЛА ЯЧЕЕК
np.random.seed(6)
nc = 49                                 # число ячеек
n = 1                                   # число спутников в ячейке
N = n * nc                              # число спутников
k = int(np.sqrt(nc))                    # корень из числа ячеек
l = 1 / k                               # длина стороны ячейки

data = np.zeros((N, 6), dtype=object)   # массив со всей информацией

# заполнение х и у
for i in range(k):
    for j in range(k):
        data[((k * i + j) * n):((k * i + j + 1) * n), 0] = np.random.random_sample(n) * l + l * j
        data[((k * i + j) * n):((k * i + j + 1) * n), 1] = np.random.random_sample(n) * l + l * i
data[:, 1] = -data[:, 1] + 1


# заполнение neighbours
mapped_data = np.zeros((N, 2))            
mapped_data[:, :] = data[:, :2]

neighbours, mapped_neighbours = [], []

for n in range(N):
    neighbours.append([])
    mapped_neighbours.append([])
    
    if data[n, 0] < 0.5:
        mapped_data[n, 0] += 1

vor = Voronoi(data[:, :2])
mapped_vor = Voronoi(mapped_data)

ridges = vor.ridge_points
mapped_ridges = mapped_vor.ridge_points

for ridge in ridges:
    neighbours[ridge[0]].append([ridge[1], 0])
    neighbours[ridge[1]].append([ridge[0], 0])

for mapped_ridge in mapped_ridges:
    mapped_neighbours[mapped_ridge[0]].append([mapped_ridge[1], 0])
    mapped_neighbours[mapped_ridge[1]].append([mapped_ridge[0], 0])
    
aoux = []
for i in range(N):
    aoux.append(neighbours[i])
print(43, neighbours[43])
print(43, mapped_neighbours[43])
print(48, neighbours[48])
print(48, mapped_neighbours[48])           
for n in range(N):
    if 0.8 < mapped_data[n, 0] < 1.2:
        for i in range(len(mapped_neighbours[n])):
            if mapped_neighbours[n][i] not in neighbours[n]:                
                #1
                if mapped_data[mapped_neighbours[n][i][0], 0] > 1:
                    mapped_neighbours[n][i][1] = -1
                else:
                    mapped_neighbours[n][i][1] = 1                    
                #2
                if not [n, -mapped_neighbours[n][i][1]] in neighbours[mapped_neighbours[n][i][0]]:
                    neighbours[mapped_neighbours[n][i][0]].append([n, -mapped_neighbours[n][i][1]])    
        neighbours[n] = mapped_neighbours[n]
print(43, mapped_neighbours[43])       
print(48, mapped_neighbours[48])           



check = 0
to_delete = []
for n in range(N):
    for i in range(len(neighbours[n])):
        if [n, -neighbours[n][i][1]] not in neighbours[neighbours[n][i][0]]:
            check += 1
            to_delete.append([n, i])
            
for i in to_delete:
    del neighbours[i[0]][i[1]]

data[:, 2] = np.random.randint(-1, 2, N)  # источник/сток/ничего
for n in range(N):
    data[n, 4] = sorted(neighbours[n], key=lambda j: j[0])
    data[n, 3] = len(data[n, 4])
#     print(n, data[n, 4])

for i in range(N):
    print(i, f'old:  {sorted(aoux[i], key=lambda j: j[0])}')
    print(i, f'new:  {sorted(mapped_neighbours[i], key=lambda j: j[0])}')
    print(i, f'last: {sorted(neighbours[i], key=lambda j: j[0])}\n')

43 [[42, 0], [48, 0], [44, 0], [36, 0], [45, 0]]
43 [[46, 0], [44, 0], [45, 0], [42, 0], [36, 0], [48, 0]]
48 [[41, 0], [47, 0], [40, 0], [46, 0], [43, 0], [45, 0]]
48 [[46, 0], [47, 0], [40, 0], [42, 0], [43, 0], [35, 0]]
43 [[46, 0], [44, 0], [45, 0], [42, 0], [36, 0], [48, 0]]
48 [[46, 0], [47, 0], [40, 0], [42, -1], [43, 0], [35, -1]]
0 old:  [[1, 0], [2, 0], [7, 0], [8, 0], [14, 0]]
0 new:  [[1, 0], [2, 0], [6, 1], [7, 0], [8, 0]]
0 last: [[1, 0], [2, 0], [6, 1], [7, 0], [8, 0]]

1 old:  [[0, 0], [2, 0], [3, 0], [6, 0]]
1 new:  [[0, 0], [2, 0], [3, 0], [6, 0], [10, 0]]
1 last: [[0, 0], [2, 0], [3, 0], [6, 0]]

2 old:  [[0, 0], [1, 0], [3, 0], [8, 0], [9, 0], [10, 0]]
2 new:  [[0, 0], [1, 0], [8, 0], [9, 0], [10, 0]]
2 last: [[0, 0], [1, 0], [3, 0], [8, 0], [9, 0], [10, 0]]

3 old:  [[1, 0], [2, 0], [4, 0], [6, 0], [10, 0]]
3 new:  [[1, 0], [4, 0], [6, 0], [11, 0], [17, 0]]
3 last: [[1, 0], [2, 0], [4, 0], [6, 0], [10, 0]]

4 old:  [[3, 0], [5, 0], [6, 0], [10, 0], [11, 0]]
4 new: 

In [126]:
connection = 0
connections = []
for n in range(N):
    for i in range(data[n, 3]):
        neib = data[n, 4][i][0]
        if n < neib:
            data[n, 4][i].append(connection)
            data[neib, 4][[q[0] for q in data[neib, 4]].index(n)].append(connection)
            connections.append([n, neib])
            bound += 1

In [103]:
print(connections)

[[0, 1], [0, 2], [0, 6], [0, 7], [0, 8], [1, 2], [1, 3], [1, 6], [2, 3], [2, 8], [2, 9], [2, 10], [3, 4], [3, 6], [3, 10], [4, 5], [4, 6], [4, 10], [4, 11], [5, 6], [5, 11], [5, 12], [6, 7], [6, 12], [6, 13], [7, 8], [7, 13], [7, 14], [7, 15], [8, 9], [8, 15], [8, 16], [9, 10], [9, 16], [9, 17], [9, 23], [10, 11], [10, 17], [11, 12], [11, 17], [11, 18], [11, 19], [12, 13], [12, 19], [13, 14], [13, 19], [13, 20], [14, 15], [14, 20], [14, 21], [14, 22], [15, 16], [15, 22], [15, 23], [16, 23], [17, 18], [17, 23], [17, 24], [17, 25], [18, 19], [18, 25], [19, 20], [19, 25], [19, 26], [19, 27], [20, 21], [20, 27], [21, 22], [21, 27], [21, 28], [21, 29], [21, 36], [22, 23], [22, 29], [23, 24], [23, 29], [24, 25], [24, 29], [24, 30], [24, 31], [25, 26], [25, 31], [25, 32], [26, 27], [26, 32], [26, 33], [27, 28], [27, 33], [27, 34], [28, 34], [28, 36], [28, 41], [29, 30], [29, 36], [29, 37], [30, 31], [30, 37], [30, 38], [30, 39], [31, 32], [31, 39], [32, 33], [32, 39], [32, 40], [33, 34], [33,

In [104]:
z = [1, 2, 3]
def kk(b, d):
    b[2] = d
print(z)
kk(z, 66)
print(z)

[1, 2, 3]
[1, 2, 66]


In [132]:
 np.random.choice([1, -1], 7)

array([ 1,  1, -1, -1,  1,  1,  1])

In [123]:
s

[-1,
 -1,
 0,
 1,
 -1,
 1,
 1,
 0,
 1,
 -1,
 -1,
 0,
 1,
 -1,
 -1,
 0,
 1,
 1,
 1,
 -1,
 -1,
 0,
 -1,
 0,
 0,
 1,
 0,
 1,
 1,
 -1,
 -1,
 0,
 0,
 0,
 0,
 -1,
 0,
 1,
 -1,
 1,
 0,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1]

In [124]:
len(s)

49